In [1]:
import os
import torch
# Set GPU device
os.environ["CUDA_VISIBLE_DEVICES"] = "1"

os.environ['http_proxy']  = 'http://192.41.170.23:3128'
os.environ['https_proxy'] = 'http://192.41.170.23:3128'

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device


device(type='cuda')

In [4]:
from langchain.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.schema import Document
from langchain.vectorstores import FAISS
from langchain.embeddings import HuggingFaceEmbeddings

# Load CV PDF
pdf_loader = PyPDFLoader("CV_Tooba_Mehboob.pdf")  # Change this to the correct file path
documents = pdf_loader.load()


# Split text into manageable chunks
text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=50)
split_docs = text_splitter.split_documents(documents)


In [3]:
pip install pypdf

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [5]:
import requests
from bs4 import BeautifulSoup

url = "https://tooba-portfolio.vercel.app/#portfolio"
response = requests.get(url)

if response.status_code == 200:
    soup = BeautifulSoup(response.text, "html.parser")
    text_data = soup.get_text()  # Extract raw text
else:
    text_data = "Portfolio could not be fetched."

# Save portfolio text as a document
portfolio_docs = [text_data]


# Convert `split_docs` and `portfolio_docs` into a list of strings (page_content)
cv_docs = [Document(page_content=d.page_content) for d in split_docs]  # Extract `page_content` from each Document
portfolio_docs = [Document(page_content=d) for d in portfolio_docs]  # Assuming `portfolio_docs` is already a list of strings

# Merge the documents
all_docs = cv_docs + portfolio_docs

# Verify the content
print(type(all_docs))  # Should be <class 'list'>
print(type(all_docs[0]))  # Each entry should be a Document
print(all_docs[0].page_content)  # Should print the text content of the first document


In [8]:
from langchain.vectorstores import FAISS
from langchain.embeddings import HuggingFaceEmbeddings  # Or OpenAIEmbeddings

# Initialize the embeddings model
embedding_model = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")

# Store in FAISS (vector database)
vector_db = FAISS.from_documents(all_docs, embedding_model)

# Save the vector store
vector_db.save_local("faiss_index")


/home/jupyter-st125404/.local/lib/python3.12/site-packages/transformers/utils/generic.py:441: FutureWarning: `torch.utils._pytree._register_pytree_node` is deprecated. Please use `torch.utils._pytree.register_pytree_node` instead.
  _torch_pytree._register_pytree_node(


.gitattributes:   0%|          | 0.00/1.23k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.5k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/612 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

train_script.py:   0%|          | 0.00/13.2k [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

/home/jupyter-st125404/.local/lib/python3.12/site-packages/transformers/utils/generic.py:309: FutureWarning: `torch.utils._pytree._register_pytree_node` is deprecated. Please use `torch.utils._pytree.register_pytree_node` instead.
  _torch_pytree._register_pytree_node(
/home/jupyter-st125404/.local/lib/python3.12/site-packages/transformers/utils/generic.py:309: FutureWarning: `torch.utils._pytree._register_pytree_node` is deprecated. Please use `torch.utils._pytree.register_pytree_node` instead.
  _torch_pytree._register_pytree_node(


In [9]:
vector_db = FAISS.load_local("faiss_index", embedding_model, allow_dangerous_deserialization=True)
retriever = vector_db.as_retriever()


In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline
import torch
from langchain.chains import RetrievalQA

# Load Falcon-7B
model_id = "tiiuae/falcon-7b-instruct"
tokenizer = AutoTokenizer.from_pretrained(model_id)

model = AutoModelForCausalLM.from_pretrained(
    model_id,
    torch_dtype=torch.float32,
    device_map="auto"
)

# Create text generation pipeline
llm_pipeline = pipeline("text-generation", model=model, tokenizer=tokenizer)




In [11]:
from langchain.llms import HuggingFacePipeline
from langchain.chains import RetrievalQA

# Wrap HuggingFace pipeline properly
llm = HuggingFacePipeline(pipeline=llm_pipeline)

# Define RetrievalQA with correct LLM
qa_chain = RetrievalQA.from_chain_type(
    llm=llm,  # ✅ Now correctly wrapped
    retriever=retriever
)


In [12]:
# Test basic retrieval
query = "What is my highest level of education?"
response = qa_chain.run(query)
print("Response:", response)


/home/jupyter-st125404/.local/lib/python3.12/site-packages/langchain_core/_api/deprecation.py:119: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(
Setting `pad_token_id` to `eos_token_id`:11 for open-end generation.
/home/jupyter-st125404/.local/lib/python3.12/site-packages/transformers/generation/utils.py:1273: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(
/home/jupyter-st125404/.local/lib/python3.12/site-packages/transformers/generation/utils.py:1281: UserWarning: Input length of input_ids is 525, but `max_length` is set to 20. This can lead to unexpected behavior. You should consider increasing `max_new_tokens`.
  warnings.warn(
The current implementation of Falcon calls `torch.scaled_dot_product_attention` directly, this w

Response: Use the following pieces of context to answer the question at the end. If you don't know the answer, just say that you don't know, don't try to make up an answer.

SSC Al-Amanah Youth Academy 
Marks obtained: 920 out of 1100
Final gradeA1 
Mother tongue(s):  URDU 
ABOUT ME 
WORK EXPERIENCE
EDUCATION AND TRAINING
LANGUAGE SKILLS

2023 
4th Position in Academics – University of Engineering & Technology, Peshawar 
2015 
1st Position in Academics – Muhammad Amir Memorial Academy 
2014 
1st in Hand Writing Competition – Muhammad Amir Memorial Academy 
2013 
Participation in Inter-school English Debate Competition – Aisha Institute of Modern Sciences for
Girls 
2012 
1st Position in Academics – Muhammad Amir Memorial Academy 
2011 
2nd Position in Academics – Muhammad Amir Memorial Academy 
2010

2010 
Good Behaviour – Muhammad Amir Memorial Academy 
2010 
3rd Position in Academics – Muhammad Amir Memorial Academy 
Organizing Member at event Kaawish - Literary and Debating Society,

## Some other Retriever and Generator

In [1]:
import fitz  # PyMuPDF

def extract_text_from_pdf(pdf_path):
    text = ""
    doc = fitz.open(pdf_path)
    for page in doc:
        text += page.get_text() + "\n"
    return text.strip()

cv_text = extract_text_from_pdf("CV_Tooba_Mehboob.pdf")
print(cv_text[:1000])  # Print first 1000 characters to verify


Tooba Mehboob 
Date of birth: 26/02/2002
 Nationality: Pakistani
 Gender: Female 
 Phone number: (+92) 3465547722 (Mobile) 
 
Email address: toobamehboob36@gmail.com 
 Address: Islamia Colony, Palosi Piran, 25130, Pakistan (Home) 
 
I am a highly motivated computer science graduate with a solid foundation in web technologies which includes HTML,
CSS, Scss, TailwindCSS, Material Tailwind, JQuery, JavaScript, ReactJS, NodeJS, NextJS, Express, MongoDB, Zustand,
Redux and API Integrations. My academic journey at University of Engineering and Technology, Peshawar, equipped
me with a robust skill set, reinforced by hands-on experiences and coursework. During my bachelor's I have done
project on Maize Seeds Species Classiﬁcation using Machine Learning. During my internship at software company, I
contributed to web development projects, gaining practical expertise in ReactJS, NodeJS, NextJS, Express and
MongoDB. Noteworthy achievements include development of HR software, SMS Tool Console, Data

In [2]:
import requests
from bs4 import BeautifulSoup

def extract_text_from_website(url):
    response = requests.get(url)
    soup = BeautifulSoup(response.text, "html.parser")
    return soup.get_text()

portfolio_text = extract_text_from_website("https://tooba-portfolio.vercel.app/")
print(portfolio_text[:1000])  # Print first 1000 characters to verify


Tooba MehboobYou need to enable JavaScript to run this app.


In [3]:
from langchain.vectorstores import FAISS
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.docstore.document import Document

# Load embedding model
embedding_model = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")

# Convert extracted text into LangChain Documents
cv_doc = Document(page_content=cv_text)
portfolio_doc = Document(page_content=portfolio_text)

# Create FAISS database
vector_db = FAISS.from_documents([cv_doc, portfolio_doc], embedding_model)

# Save vector database
vector_db.save_local("faiss_index")


/home/jupyter-st125404/.local/lib/python3.12/site-packages/transformers/utils/generic.py:441: FutureWarning: `torch.utils._pytree._register_pytree_node` is deprecated. Please use `torch.utils._pytree.register_pytree_node` instead.
  _torch_pytree._register_pytree_node(
/home/jupyter-st125404/.local/lib/python3.12/site-packages/transformers/utils/generic.py:309: FutureWarning: `torch.utils._pytree._register_pytree_node` is deprecated. Please use `torch.utils._pytree.register_pytree_node` instead.
  _torch_pytree._register_pytree_node(
/home/jupyter-st125404/.local/lib/python3.12/site-packages/transformers/utils/generic.py:309: FutureWarning: `torch.utils._pytree._register_pytree_node` is deprecated. Please use `torch.utils._pytree.register_pytree_node` instead.
  _torch_pytree._register_pytree_node(


In [9]:
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline, AutoModelForSeq2SeqLM
import torch
model_id = "google/flan-t5-large"
tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModelForSeq2SeqLM.from_pretrained(model_id)





model.safetensors:   0%|          | 0.00/3.13G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

In [ ]:
query = "Summarize my academic goals in one sentence."
input_text = f"Question: {query} Answer:"
response = model.generate(tokenizer.encode(input_text, return_tensors="pt"), max_length=100)
print(tokenizer.decode(response[0], skip_special_tokens=True))

In [12]:
def generate_answer(query, context):
    """Generate answer using FLAN-T5"""
    input_text = f"Question: {query} Context: {context} Answer:"
    inputs = tokenizer(input_text, return_tensors="pt")

    output = model.generate(**inputs, max_length=100)
    return tokenizer.decode(output[0], skip_special_tokens=True)

In [13]:
from langchain.chains import RetrievalQA
from langchain.chains.question_answering import load_qa_chain

# Load FAISS Retriever
retriever = FAISS.load_local("faiss_index", embedding_model, allow_dangerous_deserialization=True).as_retriever()

def rag_chatbot(query):
    """Retrieve relevant context and generate response"""
    retrieved_docs = retriever.get_relevant_documents(query)
    
    if not retrieved_docs:
        return "Sorry, I couldn't find relevant information."

    # Combine retrieved documents into one context
    context = " ".join([doc.page_content for doc in retrieved_docs])

    # Generate answer using FLAN-T5
    response = generate_answer(query, context)
    return response

# Test chatbot
query = "What is my highest level of education?"
response = rag_chatbot(query)
print("Chatbot:", response)


Token indices sequence length is longer than the specified maximum sequence length for this model (1928 > 512). Running this sequence through the model will result in indexing errors


Chatbot: Bachelor of Science in Computer Science


In [14]:
query = "What is my name?"
response = rag_chatbot(query)
print("Chatbot:", response)

Chatbot: Tooba Mehboob


In [15]:
query = "What are my academic achievements?"
response = rag_chatbot(query)
print("Chatbot:", response)

Chatbot: CGPA: 3.57 out of 4.00 Final grade A Thesis Maize seeds species classification using Machine Learning


In [16]:
query = "Did i studied at Al amanah youth academy?"
response = rag_chatbot(query)
print("Chatbot:", response)

Chatbot: yes


In [17]:
query = "How good I am in studies?"
response = rag_chatbot(query)
print("Chatbot:", response)

Chatbot: I am a highly motivated computer science graduate with a solid foundation in web technologies
